In [21]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd()
if PROJECT_ROOT.name == "notebooks":
    PROJECT_ROOT = PROJECT_ROOT.parent
sys.path.insert(0, str(PROJECT_ROOT))

import numpy as np
import pandas as pd

from src.vicon_utils import read_vicon_csv, find_xyz_cols

In [22]:
# Project paths

PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()

ROOT = PROJECT_ROOT / "data" / "test" / "VICON_CSV"
RESULTS_DIR = PROJECT_ROOT / "results"
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

# print("Project root:", PROJECT_ROOT)
# print("Vicon CSV root:", ROOT)
# print("ROOT exists:", ROOT.exists())
# print("Results dir:", RESULTS_DIR.resolve())

In [23]:
# Project paths

# PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()

# RESULTS_DIR = PROJECT_ROOT / "results"
# RESULTS_DIR.mkdir(parents=True, exist_ok=True)

print("Project root:", PROJECT_ROOT)
print("Vicon CSV root:", ROOT)
print("ROOT exists:", ROOT.exists())
print("Results dir:", RESULTS_DIR.resolve())

Project root: /Users/matysprecloux/Desktop/Master IEAP/Code MOTTET/Defense /SYNCOGESTM2
Vicon CSV root: /Users/matysprecloux/Desktop/Master IEAP/Code MOTTET/Defense /SYNCOGESTM2/data/test/VICON_CSV
ROOT exists: True
Results dir: /Users/matysprecloux/Desktop/Master IEAP/Code MOTTET/Defense /SYNCOGESTM2/results


In [24]:
def shoulder_width_vicon(csv_path: Path) -> dict:
    """
    Compute shoulder width (mm) for P1 and P2 from a Vicon CSV.

    Shoulder width = 3D distance between left and right shoulder markers per frame.
    We report median and mean across frames.

    Tokens:
    - P1: epaule_D / epaule_G
    - P2: 2epaule_D / 2epaule_G
    """
    df = read_vicon_csv(csv_path)
    cols = list(df.columns)

    out = {"csv": str(csv_path), "file": csv_path.name}

    for pid, prefix in [("P1", ""), ("P2", "2")]:
        tokenR = f"{prefix}epaule_D"
        tokenL = f"{prefix}epaule_G"

        RX, RY, RZ = find_xyz_cols(cols, tokenR)
        LX, LY, LZ = find_xyz_cols(cols, tokenL)

        if None in (RX, RY, RZ, LX, LY, LZ):
            out[f"{pid}_error"] = f"missing {tokenR}/{tokenL} (X/Y/Z)"
            out[f"{pid}_found"] = str([RX, RY, RZ, LX, LY, LZ])
            continue

        sw = np.sqrt((df[RX] - df[LX])**2 + (df[RY] - df[LY])**2 + (df[RZ] - df[LZ])**2)
        sw = sw.replace([np.inf, -np.inf], np.nan).dropna()

        out[f"{pid}_shoulder_width_median_mm"] = float(sw.median())
        out[f"{pid}_shoulder_width_mean_mm"]   = float(sw.mean())
        out[f"{pid}_n_frames_used"]            = int(len(sw))

    return out

In [25]:
csv_files = sorted(ROOT.rglob("*.csv"))
print("CSV found:", len(csv_files))

rows = []
for f in csv_files:
    try:
        rows.append(shoulder_width_vicon(f))
    except Exception as e:
        rows.append({"csv": str(f), "file": f.name, "error": repr(e)})

df_out = pd.DataFrame(rows)

out_path = RESULTS_DIR / "vicon_shoulder_width_by_csv.xlsx"
df_out.to_excel(out_path, index=False)

print("✅ Saved:", out_path.resolve())
df_out.head()

CSV found: 3
✅ Saved: /Users/matysprecloux/Desktop/Master IEAP/Code MOTTET/Defense /SYNCOGESTM2/results/vicon_shoulder_width_by_csv.xlsx


,csv,file,P1_shoulder_width_median_mm,P1_shoulder_width_mean_mm,P1_n_frames_used,P2_shoulder_width_median_mm,P2_shoulder_width_mean_mm,P2_n_frames_used
0,/Users/matysprecloux/Desktop/Master IEAP/Code ...,SEATEDD01.csv,281.600411,279.424604,18012,305.506314,304.438354,18012
1,/Users/matysprecloux/Desktop/Master IEAP/Code ...,SEMID01.csv,284.547768,283.576016,18000,336.097952,335.261784,18000
2,/Users/matysprecloux/Desktop/Master IEAP/Code ...,STANDINGD01.csv,290.321487,288.140682,18000,345.850784,345.675057,18000
